In [17]:
from pulp import *
import sqlite3

KCALS_GRAM_FAT = 9
KCALS_GRAM_CARBS = 4
KCALS_GRAM_PROTEIN = 4

MIN_GRAMS = 5
MAX_GRAMS = 200

MAX_PCT_FAT = 0.25
MIN_PCT_PROT = 0.4

PENALTY_PROT = 100
PENALTY_FAT = 100
PENALTY_KCALS = 10

kcals_objetive = 300

food_ids = [9, 48, 2, 53, 12]

conn = sqlite3.connect("fatsecret_scrapper/food.db")
c = conn.cursor()

foods = c.execute(
    "SELECT * FROM food WHERE id IN ({})".format(",".join("?" * len(food_ids))),
    food_ids,
).fetchall()

food_names = [f[1] for f in foods]
food_names = [f.replace(" ", "_") for f in food_names]
food_cals = [f[7] for f in foods]
food_fats = [f[8] for f in foods]
food_carbs = [f[9] for f in foods]
food_prots = [f[10] for f in foods]
gram_multiplier = [f[11] for f in foods]
gram_multiplier = [1 if g is None else g for g in gram_multiplier]

# Update food_names after replacing spaces with underscores
food_names = [f.replace(" ", "_") for f in food_names]


def print_rest(prob):
    # Print results
    print("=========================================")
    print("Status: {:>10}".format(LpStatus[prob.status]))
    if LpStatus[prob.status] == "Optimal":
        print("Objective kcals: {:>10.2f}".format(kcals_objetive) + " kcals")
        print("Target kcals: {:>13.2f}".format(value(prob.objective)) + " kcals")

        fat_kcals = 0
        prot_kcals = 0
        carb_kcals = 0

        food_vars = [(f, value(vars[f])) for f in food_names]
        food_vars_sorted = sorted(food_vars, key=lambda x: x[1])

        for f, var_value in food_vars_sorted:
            fat_kcals += var_value * food_fats[food_names.index(f)] * KCALS_GRAM_FAT
            prot_kcals += (
                var_value * food_prots[food_names.index(f)] * KCALS_GRAM_PROTEIN
            )
            carb_kcals += var_value * food_carbs[food_names.index(f)] * KCALS_GRAM_CARBS

        total_kcals = fat_kcals + prot_kcals + carb_kcals
        print("Result kcals: {:>13.2f}".format(total_kcals) + " kcals")
        print("=========================================")
        print("Macros:")
        print("=========================================")
        print(
            "Result protein: {:>10.2f}".format(prot_kcals / KCALS_GRAM_PROTEIN) + " %"
        )
        print("Result fat: {:>14.2f}".format(fat_kcals / KCALS_GRAM_FAT) + " %")
        print("Result carbs: {:>12.2f}".format(carb_kcals / KCALS_GRAM_CARBS) + " %")
        print("=========================================")
        print("Foods:")
        print("=========================================")
        for f, var_value in food_vars_sorted:
            print("- {:>10.2f}".format(var_value) + " grams of " + f.replace("_", " "))
        print("=========================================")
        print("Slack variables:")
        print("=========================================")
        print("Protein: {:>10.2f}".format(value(slack_protein)))
        print("Fat: {:>14.2f}".format(value(slack_fat)))
        print("Kcals: {:>12.2f}".format(value(slack_kcals)))


####################################################################################################

# Reset problem
prob = LpProblem("The food problem", LpMinimize)

vars = LpVariable.dicts(
    name="food",
    indices=food_names,
    lowBound=MIN_GRAMS,
    upBound=MAX_GRAMS,
    cat="Integer",
)

multipliers = LpVariable.dicts(
    "multiplier", food_names, lowBound=1, upBound=MAX_GRAMS, cat="Integer"
)
for f in food_names:
    # prob += vars[f] == 5 * multipliers[f], f"Min grams of {f}"
    prob += (
        vars[f] == gram_multiplier[food_names.index(f)] * multipliers[f],
        f"Min grams of {f}",
    )

# Slack variables for constraints
slack_protein = LpVariable("slack_protein", lowBound=0, cat="Continuous")
slack_fat = LpVariable("slack_fat", lowBound=0, cat="Continuous")
slack_kcals = LpVariable("slack_kcals", lowBound=0, cat="Continuous")

# Objective = kcals
prob += (
    lpSum([vars[f] * food_cals[i] for i, f in enumerate(food_names)])
    + PENALTY_PROT * slack_protein
    + PENALTY_FAT * slack_fat
    + PENALTY_KCALS * slack_kcals,
    "Total objective",
)
# Constraints = kcals, macros, grams
# KCALS with slack
prob += (
    lpSum([vars[f] * food_cals[i] for i, f in enumerate(food_names)]) + slack_kcals
    == kcals_objetive,
    "Max kcals",
)

# MACROS with slack
# Protein >= target
protein_constraint = lpSum([vars[f] * food_prots[i] for i, f in enumerate(food_names)])
prob += (
    protein_constraint + slack_protein
    >= MIN_PCT_PROT * kcals_objetive / KCALS_GRAM_PROTEIN,
    "Min protein",
)

# Fat <= target
fat_constraint = lpSum([vars[f] * food_fats[i] for i, f in enumerate(food_names)])
prob += (
    fat_constraint - slack_fat <= MAX_PCT_FAT * kcals_objetive / KCALS_GRAM_FAT,
    "Max fat",
)


# Solve
prob.solve()

print_rest(prob)

Status:    Optimal
Objective kcals:     300.00 kcals
Target kcals:        802.30 kcals
Result kcals:        301.46 kcals
Macros:
Result protein:      41.97 %
Result fat:          13.35 %
Result carbs:         3.36 %
Foods:
-       5.00 grams of aceite de oliva
-       5.00 grams of pan integral
-       7.00 grams of claras de huevo pasteurizadas
-      65.00 grams of huevo
-     192.00 grams of pechuga de pavo lonchas
Slack variables:
Protein:       0.00
Fat:           5.02
Kcals:         0.07


c:\Users\faguilera\miniconda3\envs\reciper\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
